In [1]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog
pd.set_option("display.max_columns", None)
pd.set_option("mode.chained_assignment", None)

import warnings
warnings.filterwarnings("ignore")
from thefuzz import fuzz
from thefuzz import process
import datetime

In [2]:
class SizeDataTransformer:
    
    def __init__(self, path):
        self.path = path
        
    def read_data(self):
        if self.path.split(".")[-1] == "xlsx":
            df = pd.read_excel(self.path, None)
            catalog = {}
            for i, j in enumerate(list(df.keys())):
                catalog.setdefault(i + 1, j)
                print(i + 1, j)
            answer = int(input("Please select the sheet"))
            df = df[catalog[answer]]
        else:
            df = pd.read_csv(self.path)
        return df
    
    @staticmethod
    def tire_type(df, category, channel=None, years=True):
        if years == True:
            df = df[(df["KA分類"] == category) & (df["大通路分類"] == channel) & (df["年月"].astype(str).str.contains("2022|2023"))]
        else:
            df = df[(df["KA分類"] == category) & (df["大通路分類"] == channel) & (~df["年月"].isin([2024, 2025, 2026]))]
        
        df["整理後的日期"] = pd.to_datetime(df["年月"], format="%Y%m")
        return df
        
        
        
    
    

In [3]:
root = tk.Tk()
root.withdraw()
file_path = filedialog.askopenfilename()
df = SizeDataTransformer(file_path)
df = df.read_data()

1 工作表2
2 樞紐 for I-SOM
3 樞紐for通路別
4 工作表1


In [4]:
result = SizeDataTransformer.tire_type(df, "TBR(G)", "REP", False)

In [5]:
result["條數"] = result["條數"].fillna(0)

In [6]:
result["舊分類"] = result["舊分類"].map(lambda x: "LSR" if "LSR" in x else x)

In [ ]:
"D:\kc.hsu\OneDrive - Bridgestone\數據\2023ACT\2023_Size Data_All 的複本.xlsx"

In [7]:
today = datetime.date.today().strftime('%Y%m%d')
path = r"D:\kc.hsu\OneDrive - Bridgestone\數據\2023ACT\{}{}".format(today, "_SizeData.xlsx")

In [8]:
result.to_excel(path, index=False)

In [14]:
result.to_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\2023ACT\20230609_power_bi_2022-2023_Size Data_All_TBR_rawling_data.xlsx", index=False)

In [ ]:
df = pd.read_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\2022ACT\202301 ISOM Size data.xlsx", None)
df.keys()

In [ ]:
df = df["工作表1"]

In [ ]:
df = df[(df["大通路分類"] == "REP") & (df["國內市場財別"] == "TBR(G)")]
df.to_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\2022ACT\202301 ISOM Size data_TBR版本.xlsx", index=False)

In [ ]:
df = pd.read_clipboard()

In [ ]:
result = df[(df["Size Designation"].str.startswith("P") == False) & (df["PrCode"].str.contains("TBR|LSR"))]

In [ ]:
bpcom_dict = dict(zip(result["PrCode"], result["Product Name"]))

In [ ]:
new_rule = pd.read_clipboard()

In [ ]:
new_rule_dict = dict(zip(new_rule["Product name"], new_rule["Category"]))

In [ ]:
product_list = new_rule["Product name"].values.tolist()

In [ ]:
isom = pd.read_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\2022ACT\202301 ISOM Size data_TBR版本.xlsx")
isom

In [ ]:
isom["Product Name"] = isom["商品代號"].map(bpcom_dict)

In [ ]:
isom["Product Name"] = isom["Product Name"].fillna("na")

In [ ]:
final_result = pd.concat([isom, pd.DataFrame(isom["Product Name"].map(lambda x: process.extractOne(x, product_list)).tolist(), columns=["匹配商品名稱", "Confidence"])], axis=1)

In [ ]:
final_result["New Category"] = final_result["匹配商品名稱"].map(new_rule_dict)

In [ ]:
final_result.to_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\2022ACT\202301 ISOM Size data_TBR版本.xlsx", index=False)

In [2]:
df = pd.read_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\2023ACT\2023_Size Data_All 的複本.xlsx", None)

In [5]:
df = df["工作表1"]

In [11]:
df = df[(df["國內市場財別"] == "TBR(G)") & (df["大通路分類"] == "REP")]

In [13]:
category_table = pd.read_excel("./ReferenceTable/20220801_category_types.xlsx")
cuscode_type = dict(zip(category_table["客戶代號"], category_table["大通路"]))

In [17]:
df["客戶大通路別"] = df["客戶代號"].map(cuscode_type)

In [22]:
df = df[df["年月"].astype(str).str.contains("2022|2023")]

In [24]:
df["修正後日期"] = pd.to_datetime(df["年月"], format="%Y%m")

In [26]:
df.to_clipboard()

In [2]:
df = pd.read_clipboard()

In [19]:
df.groupby(["month", "company_name", "tyre_type", "category", "upper_tag", "lower_tag"])["en_product_name", "items", "size", "model"].apply(list).reset_index()

,month,company_name,tyre_type,category,upper_tag,lower_tag,0
0,1110106,昆翔國際股份有限公司,輪胎,IC1,8897376,8897785,"[en_product_name, items, size, model]"


In [20]:
df.set_index(["month", "company_name", "tyre_type", "category", "upper_tag", "lower_tag"])[["en_product_name", "items", "size", "model"]].values.tolist()

[['TYRES - RADIAL 315/80R22.5 154/150M M729',
  1,
  '315/80R22.5 154/150M',
  'M729'],
 ['315/80R22.5 154/150M R150', 3, '315/80R22.5 154/150M', 'R150']]